# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import pandas as pd
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
sales = pd.read_csv('../input/kc_house_data.csv',dtype = dtype_dict)

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
from sklearn import linear_model 

In [6]:
model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, normalize=True)

Find what features had non-zero weight.

In [7]:
coef = model_all.coef_
intercept = model_all.intercept_
t = pd.DataFrame({'name':['intercept']+all_features,'value':[intercept]+coef.tolist()})

In [8]:
# this is cool
t[t.value!=0]

,name,value
0,intercept,-218136.214035
4,sqft_living,134.439314
11,view,24750.004586
13,grade,61749.103091


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [9]:
testing = pd.read_csv('../input/wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('../input/wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('../input/wk3_kc_house_valid_data.csv', dtype=dtype_dict)

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [10]:
# modifying columns as before
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

In [12]:
l1_penalty = np.logspace(1, 7, num=13)
l1_penalty

array([1.00000000e+01, 3.16227766e+01, 1.00000000e+02, 3.16227766e+02,
       1.00000000e+03, 3.16227766e+03, 1.00000000e+04, 3.16227766e+04,
       1.00000000e+05, 3.16227766e+05, 1.00000000e+06, 3.16227766e+06,
       1.00000000e+07])

In [30]:
penalties_and_rss=[]
for i in l1_penalty:
    train_model_all = linear_model.Lasso(i, normalize=True) # set parameters
    train_model_all.fit(training[all_features], training['price']) # learn weights
    y = pd.Series(train_model_all.coef_, index=all_features)
    RSS = np.sum((train_model_all.predict(validation[all_features])-validation['price'])**2)
    penalties_and_rss.append(tuple([i,RSS]))
    

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [40]:
# min penalty and rss value
min(penalties_and_rss)

(10.0, 398213327300135.0)

In [34]:
max(penalties_and_rss)

(10000000.0, 1222506859427163.0)

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [44]:
# lets use the min l2 penalty mentioned above
train2_model_all = linear_model.Lasso(alpha=min(penalties_and_rss)[0], normalize=True)

In [42]:
train2_model_all.fit(training[all_features], training['price'])

Lasso(alpha=10.0, normalize=True)

In [43]:
np.count_nonzero(train2_model_all.coef_) + np.count_nonzero(train2_model_all.intercept_)

15

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [45]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [49]:
#l1_penalty_values = np.logspace(8, 10, num=20)
l1_penalty_values = np.logspace(1, 4, num=20)


Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [62]:
counts_of_nonzeros = []
for l1_penalty in l1_penalty_values:
    model = linear_model.Lasso(l1_penalty, normalize=True) # set parameters
    model.fit(training[all_features], training['price']) # learn weights
    if model.intercept_ != 0:
        counts = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
        counts_of_nonzeros.append((l1_penalty,counts))
    else:
        counts = np.count_nonzero(model.coef_) + 1
        counts_of_nonzeros.append((l1_penalty,counts))    

In [63]:
print (counts_of_nonzeros)

[(10.0, 15), (14.38449888287663, 15), (20.6913808111479, 15), (29.76351441631318, 15), (42.81332398719393, 13), (61.58482110660264, 12), (88.58667904100822, 11), (127.42749857031335, 10), (183.29807108324357, 7), (263.6650898730358, 6), (379.26901907322497, 6), (545.5594781168514, 6), (784.7599703514607, 5), (1128.8378916846884, 3), (1623.776739188721, 3), (2335.7214690901214, 2), (3359.818286283781, 1), (4832.930238571752, 1), (6951.927961775606, 1), (10000.0, 1)]


In [64]:
list_greater = [k for k,v in counts_of_nonzeros if v > 7]

In [65]:
l1_penalty_min = max(list_greater)

In [66]:
l1_penalty_min

127.42749857031335

In [67]:
list_less = [m for m,n in counts_of_nonzeros if n < 7]
l1_penalty_max = min(list_less)
l1_penalty_max

263.6650898730358

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [68]:
counts_of_nonzeros_2 = []
RSS_list = []
for l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
    train_model_3 = linear_model.Lasso(l1_penalty, normalize=True) # set parameters
    train_model_3.fit(training[all_features], training['price']) # learn weights
    RSS = np.sum((train_model_3.predict(validation[all_features])-validation['price'])**2)
    RSS_list.append((l1_penalty,RSS))
    if train_model_3.intercept_ == 0:
        counts = np.count_nonzero(train_model_3.coef_) + np.count_nonzero(train_model_3.intercept_)
        counts_of_nonzeros_2.append((l1_penalty,counts))
    else:
        counts = np.count_nonzero(train_model_3.coef_) + 1
        counts_of_nonzeros_2.append((l1_penalty,counts))

In [70]:
max_7_list = [j for j,p in counts_of_nonzeros_2 if p == 7] 
max_7_list

[156.10909673930755,
 163.2794962815561,
 170.44989582380464,
 177.6202953660532,
 184.79069490830176,
 191.96109445055032,
 199.13149399279888]

In [71]:
RSS_list_2 = [(y,x) for x,y in RSS_list if x in max_7_list]

In [75]:
RSS_list_2.sort()

In [76]:
RSS_list_2

[(440037365263316.56, 156.10909673930755),
 (440777489641605.25, 163.2794962815561),
 (441566698090139.94, 170.44989582380464),
 (442406413188666.25, 177.6202953660532),
 (443296716874315.06, 184.79069490830176),
 (444239780526141.6, 191.96109445055032),
 (445230739842614.2, 199.13149399279888)]

In [77]:
train_model_4 = linear_model.Lasso(max_7_list[0], normalize=True) # set parameters
train_model_4.fit(training[all_features], training['price']) # learn weights

Lasso(alpha=156.10909673930755, normalize=True)

In [78]:
pd.Series(train_model_4.coef_, index = all_features)

bedrooms                -0.000000
bedrooms_square         -0.000000
bathrooms            10610.890284
sqft_living            163.380252
sqft_living_sqrt         0.000000
sqft_lot                -0.000000
sqft_lot_sqrt           -0.000000
floors                   0.000000
floors_square            0.000000
waterfront          506451.687115
view                 41960.043555
condition                0.000000
grade               116253.553700
sqft_above               0.000000
sqft_basement            0.000000
yr_built             -2612.234880
yr_renovated             0.000000
dtype: float64

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [ ]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?